(how-to-select-algorithms)=
# How to select a local optimizer

This guide explains how to choose a local optimizer that works well for your problem. 
Depending on your [strategy for global optimization](how_to_globalization.ipynb) it 
is also relevant for global optimization problems. 

## Important facts 

- There is no optimizer that works well for all problems 
- Making the right choice can lead to enormous speedups
- Making the wrong choice can mean that you cannot solve your problem at all


## The four steps for selecting algorithms

Algorithm selection is a mix of theory and experimentation. We recommend the following 
for steps:

1. Theory: Select three to 5 candidate algorithms based on the properties 
of your problem. Below we provide a simple decision tree for this step.
2. Experiments: Run the candidate algorithms fo a small number of function 
evaluations. As a rule of thumb, use between `n_params` and `10 * n_params`
evaluations. 
3. Comparison: Compare the results in a criterion plot.
4. Optimization: Re-run the optimization algorithm with the best results until 
convergence. Use the best parameter vector from the experiments as starting point.

These steps work well for most problems. Sometimes you need [variations](four-steps-variations).

## An example problem

As an example we use the Trid function. The Trid function has no local minimum except 
the global one. It is defined for any number of dimensions, we will pick 20. As starting 
values we will pick the vector [0, 1, ..., 19]. 

A Python implementation of the function and its gradient looks like this:

In [ ]:
import numpy as np

import optimagic as om


def trid_scalar(x):
    """Implement Trid function: https://www.sfu.ca/~ssurjano/trid.html."""
    return ((x - 1) ** 2).sum() - (x[1:] * x[:-1]).sum()


def trid_gradient(x):
    """Calculate gradient of trid function."""
    l1 = np.insert(x, 0, 0)
    l1 = np.delete(l1, [-1])
    l2 = np.append(x, 0)
    l2 = np.delete(l2, [0])
    return 2 * (x - 1) - l1 - l2

## Step 1: Theory

The below decision tree offers a practical guide on how to narrow down the set of algorithms to experiment with, based on the theoretical properties of your problem:

```{mermaid}
graph LR
    classDef highlight fill:#FF4500;
    A["Do you have<br/>nonlinear constraints?"] -- yes --> B["differentiable?"]
    B["differentiable?"] -- yes --> C["'ipopt', 'nlopt_slsqp', 'scipy_trust_constr'"]
    B["differentiable?"] -- no --> D["'scipy_cobyla', 'nlopt_cobyla'"]

    A["Do you have<br/>nonlinear constraints?"] -- no --> E["Can you exploit<br/>a least-squares<br/>structure?"]
    E["Can you exploit<br/>a least-squares<br/>structure?"] -- yes --> F["differentiable?"]
    E["Can you exploit<br/>a least-squares<br/>structure?"] -- no --> G["differentiable?"]

    F["differentiable?"] -- yes --> H["'scipy_ls_lm', 'scipy_ls_trf', 'scipy_ls_dogleg'"]
    F["differentiable?"] -- no --> I["'nag_dflos', 'pounders', 'tao_pounders'"]

    G["differentiable?"] -- yes --> J["'scipy_lbfgsb', 'nlopt_lbfgsb', 'fides'"]
    G["differentiable?"] -- no --> K["'nlopt_bobyqa', 'nlopt_neldermead', 'neldermead_parallel'"]
```

Let's go through the steps for the Trid function:

1. There are no nonlinear constraints our solution needs to satisfy
2. There is no least-squares structure we can exploit 
3. The function is differentiable and we have a closed form gradient that we would like 
to use. 

We therefore end up with the candidate algorithms `scipy_lbfgsb`, `nlopt_lbfgsb`, and 
`fides`.


## Step 2: Experiments

Below, we simply run optimizations with all algorithms in a loop and store the result 
in a dictionary. We limit the number of function evaluations to 8. Since some algorithms 
only support a maximum number of iterations as stopping criterion we also limit the 
number of iterations to 8.

In [ ]:
results = {}
for algo in ["scipy_lbfgsb", "nlopt_lbfgsb", "fides"]:
    results[algo] = om.minimize(
        fun=trid_scalar,
        jac=trid_gradient,
        params=np.arange(20),
        algorithm=algo,
        algo_options={"stopping_maxfun": 8, "stopping_maxiter": 8},
    )

## Step 3: Comparison

Next we plot the optimizer histories to find out which optimizer worked best:

In [ ]:
fig = om.criterion_plot(results, max_evaluations=8)
fig.show(renderer="png")

All optimizers work pretty well here and since this is a very simple problem, any of them 
would probably find the optimum in a reasonable time. However, `nlopt_lbfgsb` is a bit 
better than the others, so we will select it for the next step. 

## Step 4: Optimization 

All that is left to do is to run the optimization until convergence with the best 
optimizer. To avoid duplicated calculations, we can already start from the previously 
best parameter vector:

In [ ]:
best_x = results["nlopt_lbfgsb"].params
results["nlopt_lbfgsb_complete"] = om.minimize(
    fun=trid_scalar,
    jac=trid_gradient,
    params=best_x,
    algorithm="nlopt_lbfgsb",
)

Looking at the result in a criterion plot we can see that the optimizer converges after 
a bit more than 30 function evaluations. 

In [ ]:
fig = om.criterion_plot(results)
fig.show(renderer="png")

(four-steps-variations)=

## Variations

The four steps described above work very well in most situations. However, sometimes 
it makes sense to deviate: 

- If you are unsure about some of the questions in step 1, select more algorithms for 
the experimentation phase and run more than 1 algorithm until convergence. 
- If it is very important to find a precise optimum, run more than 1 algorithm until 
convergence. 
- If you have a very fast objective function, simply run all candidate algorithms until 
convergence. 